In [44]:
# initial setup and calculations
t0_special = 0
interval_secs = 2592000
precision = 10 ** 18
periods = 36

start_block_timestamp = 1513627200
deposits = [
    {
        "eth_address": "0xb493c9C0C0aBfd9847baB53231774f13BF882eE9",
        "face_value": 3581474393280396
    },
    {
        "eth_address": "0x0e24D8Fcdf0c319dF03998Cc53F4FBA035D9a4f9",
        "face_value": 9318691733200000
    },
    {
        "eth_address": "0x4778bE92Dd5c51035bf80Fca564ba5E7Fad5FB6d",
        "face_value": 2795607519960000
    },
    {
        "eth_address": "0x8567462b8E8303637F0004B2E664993314e58BD7",
        "face_value": 2795607519960000
    }
]

total_fv = sum([a["face_value"] for a in deposits])
total = 30489196119340505

# does not have the extra guards
def period_at(timestamp):
    return ((timestamp - start_block_timestamp) // interval_secs) + 1;

def fraction_withdraw_at(timestamp):
    return ((t0_special + period_at(timestamp)) * precision) // (t0_special + periods)

def withdraw_at_timestamp(deposit, total, total_fv, timestamp):
    frac = fraction_withdraw_at(timestamp)
    return ((deposit * frac * total) // total_fv) // precision

def total_withdrawal(deposit, timestamp):
    return withdraw_at_timestamp(deposit, total, total_fv, timestamp)

This is the total withdrawn as is on December 1st, meaning the last payout we were scheduled to do (Nov) has already been done:

In [45]:
# after november timestamp (dec 01)
dec_01 = 1543622400

# note that this calculation is incorrect, as it doesn't account for the scaling increase for each account
total_withdrawn = 0
total_fv_remaining = 0
for i in deposits:
    withdrawn = total_withdrawal(i["face_value"], dec_01)
    total_withdrawn += withdrawn
    total_fv_remaining += i["face_value"] - withdrawn
    print("address:", i["eth_address"],
          "total:",i["face_value"],
          "withdrawn:", withdrawn, 
          "remaining:", i["face_value"] - withdrawn)
    
print("total remaining in contract:\t", total - total_withdrawn)
print("total face_value remaining:\t", total_fv_remaining)

address: 0xb493c9C0C0aBfd9847baB53231774f13BF882eE9 total: 3581474393280396 withdrawn: 1968417505620333 remaining: 1613056887660063
address: 0x0e24D8Fcdf0c319dF03998Cc53F4FBA035D9a4f9 total: 9318691733200000 withdrawn: 5121654917183230 remaining: 4197036816016770
address: 0x4778bE92Dd5c51035bf80Fca564ba5E7Fad5FB6d total: 2795607519960000 withdrawn: 1536496475154969 remaining: 1259111044805031
address: 0x8567462b8E8303637F0004B2E664993314e58BD7 total: 2795607519960000 withdrawn: 1536496475154969 remaining: 1259111044805031
total remaining in contract:	 20326130746227004
total face_value remaining:	 8328315793286895


After Discussing with Jamal, this is the conclusion. The numbers seem off compared to his excel spreadsheet because we have scaled these calculations with a bonus. This needs to be factored in for the calculations of the values on Aion.

In [46]:
eth_trs_bonus = total - total_fv
print("trs bonus allocation:", eth_trs_bonus)

bonus_percentage_increase = total / total_fv
print("percentage increase over face value: ", bonus_percentage_increase)

trs bonus allocation: 11997814952940109
percentage increase over face value:  1.6488328181099114


In [47]:
# scaled initial deposits
def scale_face(fv):
    return ((fv * total * precision) // total_fv) // precision
    
for i in deposits:
    i["scaled_face_value"] = scale_face(i["face_value"])
    print("address:", i["eth_address"],
          "original:", i["face_value"], 
          "scaled:", i["scaled_face_value"])
    
print("total:", total);
print("scaled total:", sum([i["scaled_face_value"] for i in deposits]))

address: 0xb493c9C0C0aBfd9847baB53231774f13BF882eE9 original: 3581474393280396 scaled: 5905252516861000
address: 0x0e24D8Fcdf0c319dF03998Cc53F4FBA035D9a4f9 original: 9318691733200000 scaled: 15364964751549690
address: 0x4778bE92Dd5c51035bf80Fca564ba5E7Fad5FB6d original: 2795607519960000 scaled: 4609489425464907
address: 0x8567462b8E8303637F0004B2E664993314e58BD7 original: 2795607519960000 scaled: 4609489425464907
total: 30489196119340505
scaled total: 30489196119340504


So rerunning the calculations before, with these scaled values:

In [48]:
total_fv = sum([a["scaled_face_value"] for a in deposits])
total = 30489196119340505

# after november timestamp (dec 01)
dec_01 = 1543622400

# note that this calculation is incorrect, as it doesn't account for the scaling increase for each account
total_withdrawn = 0
total_fv_remaining = 0
for i in deposits:
    withdrawn = total_withdrawal(i["scaled_face_value"], dec_01)
    total_withdrawn += withdrawn
    total_fv_remaining += i["scaled_face_value"] - withdrawn
    i["scaled_remaining"] = i["scaled_face_value"] - withdrawn
    i["final_remaining"] = i["scaled_remaining"]
    print("address:", i["eth_address"],
          "total:",i["scaled_face_value"],
          "withdrawn:", withdrawn, 
          "remaining:", i["scaled_remaining"])
    
print("total remaining in contract:\t", total - total_withdrawn)
print("total face_value remaining:\t", total_fv_remaining)

address: 0xb493c9C0C0aBfd9847baB53231774f13BF882eE9 total: 5905252516861000 withdrawn: 1968417505620333 remaining: 3936835011240667
address: 0x0e24D8Fcdf0c319dF03998Cc53F4FBA035D9a4f9 total: 15364964751549690 withdrawn: 5121654917183230 remaining: 10243309834366460
address: 0x4778bE92Dd5c51035bf80Fca564ba5E7Fad5FB6d total: 4609489425464907 withdrawn: 1536496475154969 remaining: 3072992950309938
address: 0x8567462b8E8303637F0004B2E664993314e58BD7 total: 4609489425464907 withdrawn: 1536496475154969 remaining: 3072992950309938
total remaining in contract:	 20326130746227004
total face_value remaining:	 20326130746227003


Lastly, we must factor in that we are essentially netting out a balance of around 197k per month:

In [50]:
balance_transfer = 19768067000000

# this is crappy coding
balance_net = balance_transfer * 24

# arbitrarily decide that the third index offset gets the balance from the second
deposits[2]["final_remaining"] += balance_net
deposits[3]["final_remaining"] -= balance_net

for i in deposits:
    print("address:", i["eth_address"],
          "remaining:", i["final_remaining"])

address: 0xb493c9C0C0aBfd9847baB53231774f13BF882eE9 remaining: 3936835011240667
address: 0x0e24D8Fcdf0c319dF03998Cc53F4FBA035D9a4f9 remaining: 10243309834366460
address: 0x4778bE92Dd5c51035bf80Fca564ba5E7Fad5FB6d remaining: 4021860166309938
address: 0x8567462b8E8303637F0004B2E664993314e58BD7 remaining: 2124125734309938
